# Integrate IBM Process Mining data into IBM Cognos Analytics

In most cases, our clients will be using the Dashboard (Analytics) capabilities of IBM Process Mining to broadcast relevant process mining information to managers and employees.

They are also typically using extensively a Business Intelligence solution, such as IBM Cognos Analytics, and they can consider that apart from business analysts and process owners, regular employees who are participating in the business process execution should preferably keep using the enterprise BI solution.

The following python code shows how we can extract relevant information from IBM Process Mining, and how we can upload this information into IBM Cognos Analytics to create appealing dashboards.

The process is the following:

- get information from IBM Process Mining by using the IPMClient python library. The information retrieved is a json dict.
- transform the json dict into a Pandas dataframe. Sometimes, we need to flatten the json dict and to rename/transform some columns.
- save the resulting transformation into a CSV file
- upload the CSV file into IBM Cognos Analytics by using the cognos python library

## Get the data from Process Mining


In [2]:
import IPMClient as ipm
import pandas as pd
import json

ipmConfigFilename = './IPMConfig.json'
with open(ipmConfigFilename, 'r') as file:
    ipmConfig = json.load(file)  

ipmClient = ipm.Client(ipmConfig['url'], ipmConfig['userid'], ipmConfig['apikey'])
ipmProject = ipmClient.getProjectByName('Bank Account Closure')    
stats = ipmProject.retrieveModelStatistics()
processStats = ipmProject.getProcessStatistics(stats)
processStats_df = pd.json_normalize(processStats)
processStats_df.keys()
processStats_df = processStats_df[['minThroughputTime',
       'maxThroughputTime', 'avgThroughputTime', 'stdThroughputTime',
       'medianThroughputTime', 'minArrivalRate', 'maxArrivalRate',
       'avgArrivalRate', 'minTime', 'maxTime', 'filteredCases', 'filteredEvents','totalCases', 'totalEvents']]
processStats_df.to_csv('processStats.csv', index=None)

activityStats = ipmProject.getActivityStatistics(stats)
activityStats_df = pd.json_normalize(activityStats)
activityStats_df = activityStats_df[['activityName', 'statistics.frequency', 'statistics.avgDuration', 'statistics.medianDuration', 'statistics.minDuration', 'statistics.maxDuration', 'statistics.caseRepetition', 'statistics.avgRepetition', 'statistics.overallCost']]
activityStats_df.rename({'statistics.frequency': 'frequency', 'statistics.avgDuration':'avgDuration', 'statistics.medianDuration':'medianDuration', 
                         'statistics.minDuration':'minDuration', 'statistics.maxDuration':'maxDuration', 'statistics.caseRepetition':'caseRepetition', 
                         'statistics.avgRepetition':'avgRepetition', 'statistics.overallCost':'overallCost'}, axis='columns', inplace=True)
activityStats_df.to_csv('activityStats.csv', index=None)

transitionStats = ipmProject.getTransitionStatistics(stats)
transitionStats_df = pd.json_normalize(transitionStats)
transitionStats_df.keys()
transitionStats_df=transitionStats_df[['sourceActivity', 'targetActivity', 'statistics.frequency', 'statistics.avgDuration', 'statistics.medianDuration',
                                       'statistics.minDuration','statistics.maxDuration', 'statistics.parallelFrequency','statistics.caseRepetition','statistics.avgRepetition']]
transitionStats_df.rename({'statistics.frequency':'frequency', 'statistics.avgDuration':'avgDuration', 'statistics.medianDuration':'medianDuration',
                                       'statistics.minDuration':'minDuration','statistics.maxDuration':'maxDuration', 
                                       'statistics.parallelFrequency':'parallelFrequency','statistics.caseRepetition':'caseRepetition',
                                       'statistics.avgRepetition':'avgRepetition'}, axis='columns', inplace=True)
transitionStats_df.to_csv('transitionStats.csv', index=None)

dashboard = ipmProject.getDashboardByName('activity stats')
widget = dashboard.getWidgetByName('activities')
values = widget.retrieveValues()
values_df = pd.DataFrame(values)
values_df.to_csv('acticityStatsFromWidget.csv', index=None)

# RETRIEVE DATA USING PSEUDO SQL. Avoid being limited by the number of rows in a widget

query = "SELECT CASEID, leadtime, casecost(), CLOSURE_TYPE, CLOSURE_REASON, COUNTACTIVITIES, COUNTREWORKS FROM EVENTLOG GROUP BY CASEID"
#res = ipmProject.retrieveFromSQL(query)
headers = ipmClient.getHeaders()
headers['content-type'] = 'application/x-www-form-urlencoded'
data = "params={'query': '%s'}" % query
res = ipmProject.sendPostRequest(
            url=f"{ipmProject.getURL()}/analytics/integration/{ipmProject.key}/query",
            verify=ipmProject.verify,
            params={'org' : ipmProject.orgkey},
            headers=headers,
            data=data,
            files=None,
            functionName='retrieve from SQL'
        )

df = pd.DataFrame(res)



df = pd.DataFrame(res)
df.columns = ['caseid', 'leadtime', 'cost', 'closure_type', 'closure_reason', 'count_activities', 'count_reworks']
df.reindex()
df['cost'] = df['cost'].apply(lambda x: x[0])
df.to_csv('completedCases.csv', index=None)


--Process Mining: REST API called for: get token : status code : 200
--Process Mining: REST API called for: retrieve projects : status code : 200
--Process Mining: REST API called for: retrieve model-statistics : status code : 200
--Process Mining: REST API called for: retrieve dashboards : status code : 200


AttributeError: 'Project' object has no attribute 'organization'

## Upload the data into IBM Cognos Analytics

The configuration file CognosAnalytics.json includes the URL of the Cognos Analytics api, and how you want to login to create a session.
- using the API Login Key
- using the namespace / username / password


In [ ]:
# Example of credential dict in './CognosAnalytics.json'

{
    "url": "http://myCognosAnalytics:9300/api/v1/", 
    "CAMAPILoginKey": "AWlDNzY5MTJCQzg1MDU0NTQ5OTlFM0Y4ODQ5MzA5NTY4RXiiziKhbFA4nCbGAI0I+AAAAAA",
    "CAMNamespace": "",
    "CAMUsername": "", 
    "CAMPassword":"" 
}

In [ ]:
import CognosAnalyticsClient as cog
import json
class dictToObj ():
    def __init__(self, aDict):
        self.__dict__.update(aDict)

cognosConfigFilename = './CognosAnalytics.json'
with open(cognosConfigFilename, 'r') as file:
    cognosConfig = json.load(file)


cognosCredentials =  cog.cognosCreateCredentials(cognosConfig)    
auth = cog.cognosCreateSession(cognosConfig['url'], credentials=cognosCredentials)
cog.cognosUploadFile(cognosConfig['url'], auth['authkey'], auth['authvalue'], filename='processStats.csv', append=False, silent=False)
cog.cognosUploadFile(cognosConfig['url'], auth['authkey'], auth['authvalue'], filename='activityStats.csv', append=False, silent=False)
cog.cognosUploadFile(cognosConfig['url'], auth['authkey'], auth['authvalue'], filename='transitionStats.csv', append=False, silent=False)
cog.cognosUploadFile(cognosConfig['url'], auth['authkey'], auth['authvalue'], filename='completedCases.csv', append=False, silent=False)
cog.cognosUploadFile(cognosConfig['url'], auth['authkey'], auth['authvalue'], filename='activityStatsFromWidget.csv', append=False, silent=False)